# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# imports


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
import tensorflow.keras
from tensorflow.keras import models, layers, optimizers
from keras.preprocessing.image import ImageDataGenerator

from	tensorflow.keras.models import	Sequential
from	tensorflow.keras.layers import	Dense,	Activation
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Conv3D
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import train_test_split, KFold

# Print Config


In [3]:
#comment to turn off, prints more data.
def show_allData(all):
  if all:
    np.set_printoptions(threshold=np.inf) #numpy show whole array
    pd.set_option('display.max_rows', None) #pandas show whole dataframe
  else:
    np.set_printoptions(threshold=6) #numpy show whole array
    pd.set_option('display.max_rows', 6)

# Read CSV

In [4]:
# Folder path of the data
folder="/content/drive/Shareddrives/Team 8- Neural Network/Final Project/ECBE329_Project/ECBE329_Project"


## Parse Data


In [5]:
def get_data(rescale, img_size, class_mode, color_mode, zoom_range, horizontal_flip, 
             vertical_flip, validation_split,batch_size):
    """
    Function to read data from folder data path and returns 
    training and valid data
    Note: class_mode is default to sparse for this data set

    :param img_size:  Tuple of integers (height, width), defaults 
                      to (256, 256). The dimensions to which all 
                      images found will be resized.

    :param color_mode:"grayscale" or "rgb"

    :param zoom_range: Float or [lower, upper]. Range for random zoom. If a
                         float, [lower, upper] = [1-zoom_range, 1+zoom_range].

    :param horizontal_flip:  Boolean. Randomly flip inputs horizontally.

    :param vertical_flip:  Boolean. Randomly flip inputs vertical.

    :param validation_split: Float. Fraction of images reserved for validation 
                            (strictly between 0 and 1)
    :return: Tuple. training data, validation data.
    """
    # class_mode="sparse"

    datagen = ImageDataGenerator(
            rescale=rescale,
            zoom_range=zoom_range,
            shear_range=0.2,
            horizontal_flip=horizontal_flip,
            vertical_flip=vertical_flip,
            validation_split=validation_split)

    train_generator = datagen.flow_from_directory(
                      folder,
                      target_size=img_size,
                      class_mode=class_mode,
                      color_mode=color_mode,
                      batch_size=batch_size,
                      subset='training') # set as training data
    
    validation_generator = datagen.flow_from_directory(
                          folder, # same directory as training data
                          target_size=img_size,
                          class_mode=class_mode,
                          color_mode=color_mode,
                          batch_size=batch_size,
                          subset='validation') # set as validation data
    
    return train_generator, validation_generator
                              

# **Note:**
For `class_mode` in `get_data()` we need to use the specific `loss` for `compiler()` which is listed below.

Data   --->  Model Compiler


---
`class_mode` --> `loss` 
*   "sparse-->"sparse_categorical_crossentropy"
*   "categorical"-->"categorical_crossentropy"

## **Selecting Optimizer:**
We will select each optimizer to test our models
* **`SGD`**: very basic and is seldom used now. One problem is with the global learning rate associated with the same. Hence it doesn't work well when the parameters are in different scales since a low learning rate will make the learning slow while a large learning rate might lead to oscillations.
* **`RMSprop`**
* **`Adam`** : combines the good properties of Adadelta and RMSprop and hence tend to do better for most of the problems.
* **`Adagrad`** : penalizes the learning rate too harshly for parameters which are frequently updated and gives more learning rate to sparse parameters,
* **`Adamax`**: is a variant of Adam based on the infinity norm
* **`Nadam`**:is Adam RMSprop with Nesterov momentum.

## Get data
To use rgb specify,

`color_mode`=`"rgb"`

`input_shape = (img_height,img_weidth,3)`


To use grayscale specify 

`color_mode`= `"grayscale"` 

`input_shape = (img_height,img_weidth,1)`



# Model 1

## Model 1 Data

In [6]:
img_height=384
img_weidth=512

img_size=(img_height,img_weidth)

epochs=100
batch_size=64 

train_generator,validation_generator = get_data(rescale=1./255,
                                                img_size=img_size,
                                                color_mode= "grayscale",
                                                class_mode="categorical",
                                                batch_size=batch_size,
                                                zoom_range=0.2, 
                                                horizontal_flip=True, 
                                                vertical_flip=False, 
                                                validation_split=0.2)
# n_image=train_generator.samples
input_shape = (img_height,img_weidth,1)

Found 1816 images belonging to 5 classes.
Found 454 images belonging to 5 classes.


## Model Structure

In [7]:
# build the model object
model = Sequential()

# CONV_1: add CONV layer with RELU activation and depth = 32 kernels
#padding="same"
model.add(Conv2D(25, kernel_size=(5, 5), padding='valid',activation='relu',input_shape=input_shape))

# POOL_1: downsample the image to choose the best features 
model.add(MaxPooling2D(pool_size=(2,2))) #pooling

model.add(Conv2D(40, kernel_size=(5, 5), padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


# flatten since too many dimensions, we only want a classification output
model.add(Flatten())

# FC_1: fully connected to get all relevant data
model.add(Dense(100, activation='relu'))

# output a softmax to squash the matrix into output probabilities for the 10 classes
model.add(Dense(5, activation='softmax'))

model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 380, 508, 25)      650       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 190, 254, 25)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 186, 250, 40)      25040     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 93, 125, 40)       0         
_________________________________________________________________
flatten (Flatten)            (None, 465000)            0         
_________________________________________________________________
dense (Dense)                (None, 100)               46500100  
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 5

## Compile

In [8]:
# compile the model
model.compile(loss='categorical_crossentropy', 
              optimizer='Nadam', 
              metrics=['accuracy']) 


## Train model 1

In [9]:
history=model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples // batch_size,
        validation_data = validation_generator, 
        validation_steps = validation_generator.samples // batch_size,
        epochs = epochs)

Epoch 1/100
28/28 [==============================] - 42s 1s/step - loss: 10.3879 - accuracy: 0.2502 - val_loss: 1.5880 - val_accuracy: 0.2500
Epoch 2/100
28/28 [==============================] - 36s 1s/step - loss: 1.6892 - accuracy: 0.2411 - val_loss: 1.5801 - val_accuracy: 0.2522
Epoch 3/100
28/28 [==============================] - 36s 1s/step - loss: 1.7828 - accuracy: 0.2285 - val_loss: 1.5263 - val_accuracy: 0.2835
Epoch 4/100
28/28 [==============================] - 36s 1s/step - loss: 1.5636 - accuracy: 0.2881 - val_loss: 1.4805 - val_accuracy: 0.3438
Epoch 5/100
28/28 [==============================] - 36s 1s/step - loss: 1.4449 - accuracy: 0.3565 - val_loss: 1.6269 - val_accuracy: 0.2723
Epoch 6/100
28/28 [==============================] - 36s 1s/step - loss: 1.5047 - accuracy: 0.3501 - val_loss: 1.4824 - val_accuracy: 0.3504
Epoch 7/100
28/28 [==============================] - 36s 1s/step - loss: 1.4147 - accuracy: 0.3886 - val_loss: 1.5776 - val_accuracy: 0.3147
Epoch 8/100


# web model


In [10]:
img_height=384
img_weidth=512

img_size=(img_height,img_weidth)

epochs=100
batch_size=64 

train_generator,validation_generator = get_data(rescale=1./255,
                                                img_size=img_size,
                                                color_mode= "grayscale",
                                                class_mode="sparse",
                                                batch_size=batch_size,
                                                zoom_range=0.2, 
                                                horizontal_flip=True, 
                                                vertical_flip=False, 
                                                validation_split=0.2)
# n_image=train_generator.samples
input_shape = (img_height,img_weidth,1)

Found 1816 images belonging to 5 classes.
Found 454 images belonging to 5 classes.


In [11]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [12]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5,activation='softmax'))

In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_generator, epochs=epochs, 
                    validation_data=validation_generator)

Epoch 1/100
29/29 [==============================] - 43s 1s/step - loss: 7.2653 - accuracy: 0.2071 - val_loss: 1.5347 - val_accuracy: 0.3194
Epoch 2/100
29/29 [==============================] - 37s 1s/step - loss: 1.5929 - accuracy: 0.2887 - val_loss: 1.5639 - val_accuracy: 0.2952
Epoch 3/100
29/29 [==============================] - 37s 1s/step - loss: 1.4936 - accuracy: 0.3124 - val_loss: 1.5225 - val_accuracy: 0.2996
Epoch 4/100
29/29 [==============================] - 37s 1s/step - loss: 1.4410 - accuracy: 0.3709 - val_loss: 1.5336 - val_accuracy: 0.3370
Epoch 5/100
29/29 [==============================] - 37s 1s/step - loss: 1.4205 - accuracy: 0.3913 - val_loss: 1.5281 - val_accuracy: 0.3128
Epoch 6/100
29/29 [==============================] - 37s 1s/step - loss: 1.3546 - accuracy: 0.4421 - val_loss: 1.5459 - val_accuracy: 0.3282
Epoch 7/100
29/29 [==============================] - 37s 1s/step - loss: 1.3125 - accuracy: 0.4455 - val_loss: 1.5223 - val_accuracy: 0.2930
Epoch 8/100
2

# web model 2


In [22]:
img_height=384
img_weidth=512

img_size=(img_height,img_weidth)

epochs=100
batch_size=64 

train_generator,validation_generator = get_data(rescale=1./255,
                                                img_size=img_size,
                                                color_mode= "grayscale",
                                                class_mode="sparse",
                                                batch_size=batch_size,
                                                zoom_range=0.2, 
                                                horizontal_flip=True, 
                                                vertical_flip=False, 
                                                validation_split=0.2)
# n_image=train_generator.samples
input_shape = (img_height,img_weidth,1)

Found 1816 images belonging to 5 classes.
Found 454 images belonging to 5 classes.


In [23]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='sigmoid', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))

In [24]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='sigmoid'))
model.add(layers.Dense(5))

In [25]:
model.compile(optimizer='Adamax',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_generator, epochs=epochs, 
                    validation_data=validation_generator)

Epoch 1/100
29/29 [==============================] - 39s 1s/step - loss: 2.3739 - accuracy: 0.1936 - val_loss: 2.1490 - val_accuracy: 0.2115
Epoch 2/100
29/29 [==============================] - 37s 1s/step - loss: 2.0706 - accuracy: 0.2171 - val_loss: 1.9574 - val_accuracy: 0.2115
Epoch 3/100
29/29 [==============================] - 37s 1s/step - loss: 1.9196 - accuracy: 0.2133 - val_loss: 1.8253 - val_accuracy: 0.2115
Epoch 4/100
29/29 [==============================] - 37s 1s/step - loss: 1.7762 - accuracy: 0.2229 - val_loss: 1.7236 - val_accuracy: 0.2115
Epoch 5/100
29/29 [==============================] - 37s 1s/step - loss: 1.7183 - accuracy: 0.2200 - val_loss: 1.6515 - val_accuracy: 0.2115
Epoch 6/100
29/29 [==============================] - 37s 1s/step - loss: 1.6360 - accuracy: 0.2308 - val_loss: 1.6168 - val_accuracy: 0.2511
Epoch 7/100
29/29 [==============================] - 37s 1s/step - loss: 1.6092 - accuracy: 0.2522 - val_loss: 1.6024 - val_accuracy: 0.2511
Epoch 8/100
2